In [2]:

import re

import os

import time
from concurrent.futures import ThreadPoolExecutor
import time

import pandas as pd
import numpy as np
import konlpy.tag
import nltk
import matplotlib.pyplot as plt

from konlpy.tag import Komoran
from konlpy.tag import Twitter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split

ImportError: No module named 'konlpy'

In [2]:
%%time
## 리스트를 생성하고 전체, 장르별 영화평점 dataframe 저장
dflst = []
dflst.append(pd.read_csv('merged_df/merged_comments.csv'))
MvDf = pd.read_csv('MvDf.csv')

#장르명 전처리
genre = []
for i in MvDf['Genre']:
    if i =='멜로/애정/로맨스':
        genre.append('멜로애정로맨스')
    else:
        genre.append(i)
MvDf['newGenre'] = genre

gr = list(set(MvDf['newGenre']))
gr.remove(np.nan) # nan 제거
gr2 = []
for i in gr:
    try:
        dflst.append(pd.read_csv('merged_df/'+i+'merged_comments.csv',engine='python',encoding='UTF8'))
        gr2.append(i)
    except:
        1==1
gr3 = ['전체']+gr2

CPU times: user 6.28 s, sys: 334 ms, total: 6.61 s
Wall time: 6.7 s


In [3]:
len(gr3)
print(gr3)

['전체', '미스터리', '판타지', '공연실황', '블랙코미디', '액션', '무협', '전쟁', '멜로애정로맨스', '코미디', '뮤지컬', '애니메이션', '드라마', '서스펜스', '공포', '범죄', '모험', '에로', 'SF', '스릴러', '다큐멘터리', '가족']


In [4]:
print(dflst[0].shape)
dflst[0].head()

(773123, 5)


,Unnamed: 0,0,1,2,3
0,0,168037,13726146,10,'마지막 장면 진짜 존나 간지'
1,1,168037,13728402,10,'도스툼 장군님...당신은 도덕책'
2,2,168037,13726797,10,'염력보려다가 평점보고 12솔져스 봤는데 진짜 최고의 선택! 알게 된것도 많고 너무...
3,3,168037,13724265,10,"'미국 찬양하는 건 잘 모르겠고, 햄식이 잘생긴 거, 그리고 스케일 쩌는건 알겠더라..."
4,4,168037,13728590,10,'마지막에 악수할때 진짜 멋있었다 햄식이형이 말타고 총쏘는게 존멋'


In [5]:
##평점분포 확인
hist = []
for i in range(1,11):
    hist.append(sum(dflst[0]['2']==i))
x= [1,2,3,4,5,6,7,8,9,10]
plt.bar(x,hist)
plt.show()

NameError: name 'plt' is not defined

In [ ]:
## funtion which return logistic regresson , naive bayes , stochastic gradient descending acc_rate 
def getacc(df):
    scores = tuple(df.iloc[:,3])
    texts = tuple(df.iloc[:,4])

    filtered_texts = []
    filtered_labels = []

    for text, score in zip(texts, scores):
        if 3 <= score < 8:
            continue

        # 평점 기준으로 문서에 label을 부여
        # 1 ~ 4 -> 부정, 0
        # 8 ~ 10 -> 긍정, 1
        filtered_texts.append(text)
        filtered_labels.append(1 if score >= 8 else 0)

    # train test split
    train_texts, test_texts, train_labels, test_labels = train_test_split(filtered_texts, filtered_labels, test_size=0.3, random_state=0)

    def tfidf_extractor(corpus):
        vectorizer = TfidfVectorizer(min_df=1, 
                                     norm='l2',
                                     smooth_idf=True,
                                     use_idf=True,
                                     ngram_range=(1,1))
        features = vectorizer.fit_transform(corpus)
        return vectorizer, features

    #tf_idf matrix 사용
    tfidf_vectorizer, train_tfidf_features = tfidf_extractor(train_texts)
    test_tfidf_features = tfidf_vectorizer.transform(test_texts)

    lr = LogisticRegression(C=0.1, penalty='l1', random_state=0)
    lr.fit(train_tfidf_features, train_labels)
    lr_pred_labels = lr.predict(test_tfidf_features)

    MNB=MultinomialNB()
    MNB.fit(train_tfidf_features, train_labels) 
    nb_pred_labels = MNB.predict(test_tfidf_features)

    sgd=SGDClassifier(random_state = 0)
    sgd.fit(train_tfidf_features, train_labels) 
    sgd_pred_labels = sgd.predict(test_tfidf_features)
    
    ##너무 느린 알고리즘

    '''
    %%time
    knn=KMeans(n_clusters=2)
    knn.fit(train_tfidf_features, train_labels) 
    pred_labels = knn.predict(test_tfidf_features)
    print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
    print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))
    '''

    ## 2h 43m accuracy 0.86
    '''
    %%time
    from sklearn.ensemble import RandomForestClassifier
    RFT = RandomForestClassifier(n_estimators =100)
    RFT.fit(train_tfidf_features, train_labels) 
    pred_labels = RFT.predict(test_tfidf_features)
    print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
    print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))
    '''

    '''
    %%time
    from sklearn.svm import SVC
    sinvecma = SVC()
    sinvecma.fit(train_tfidf_features, train_labels) 
    pred_labels = sinvecma.predict(test_tfidf_features)
    print('Misclassified samples: {} out of {}'.format((pred_labels != test_labels).sum(),len(test_labels)))
    print('Accuracy: %.2f' % accuracy_score(test_labels, pred_labels))
    '''
    
    return(accuracy_score(test_labels, lr_pred_labels) , accuracy_score(test_labels, nb_pred_labels),accuracy_score(test_labels, sgd_pred_labels),len(train_labels),len(test_labels))

In [6]:
%%time
## make the acc_rate table and save csv
tmp = []
for i in range(len(gr3)):
    try:
        tm2 = getacc(dflst[i])
        tmp.append([gr3[i],tm2[0],tm2[1],tm2[2],tm2[3],tm2[4],tm2[3]+tm2[4]])
    except:
        1==1
result= pd.DataFrame(tmp).copy()
result.columns = ['Genre','lr_acc','nb_acc','sgd_acc','train_n_row','test_n_row','all_n_row']
result.to_csv('result.csv',encoding='CP949')

ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements

In [82]:
prop_df = pd.DataFrame()
prop_df['Genre'] = result['Genre']
prop_df['n'] = result['test_n_row']
prop_df['p'] = result.iloc[:,[1,2,3]].max(axis=1)

In [86]:
prop_df.to_csv('Rcode/prop_test.csv',encoding='UTF8',index=False)